<a href="https://colab.research.google.com/github/AURORARISE/MSSP-6070/blob/main/Week_12_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
from scipy import stats
df = pd.read_csv("/content/drive/MyDrive/MSSP6070/c. Modules/Week_12/Student Performance Data.csv")
df.columns = [c.strip().lower().replace(' ', '_').replace('/', '_') for c in df.columns]
print("Column name:")
print(df.columns)

Column name:
Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
       'math_score', 'reading_score', 'writing_score', 'total_score',
       'test_preparation_course', 'admission_prospect'],
      dtype='object')


In [42]:
df['gender'] = df['gender'].astype(str).str.strip().str.lower()

if 'test_preparation_course' in df.columns:
    df['test_preparation_course'] = df['test_preparation_course'].astype(str).str.strip().str.lower()
else:
    raise ValueError("There is no test_preparation_comse column in the data. Please use print (df. columns) to check the column names.")
numeric_cols = ['math_score', 'reading_score', 'writing_score']
print("\nNumerical column check:")
for col in numeric_cols:
    print(col, "in columns?", col in df.columns)
df['total_score'] = df[numeric_cols].mean(axis=1)
print("\nData Preview:")
print(df[['gender', 'test_preparation_course', 'total_score']].head())


Numerical column check:
math_score in columns? True
reading_score in columns? True
writing_score in columns? True

Data Preview:
   gender test_preparation_course  total_score
0  female                    none    72.666667
1  female               completed    82.333333
2  female                    none    92.666667
3    male                    none    49.333333
4    male                    none    76.333333


In [43]:
group1 = df[(df['gender'] == 'male') & (df['test_preparation_course'] == 'completed')]['total_score']
group2 = df[~((df['gender'] == 'male') & (df['test_preparation_course'] == 'completed'))]['total_score']
print("Sample size of group1 (male+completed):", len(group1))
print("Sample size for group2 (others):", len(group2))

t_stat, p_val = stats.ttest_ind(group1, group2, equal_var=False)

print("\nQ2: Men completing the prep exam vs others")
print("T-statistic:", t_stat)
print("P-value:", p_val)

alpha = 0.05
if p_val < alpha:
    print("Conclusion: At a significance level of 0.05, there is a statistically significant difference in the average scores of males completing the prep exam compared to other students.")
else:
    print("Conclusion: At a significance level of 0.05, no significant difference was found between male students who completed the prep exam and other students.")

Sample size of group1 (male+completed): 174
Sample size for group2 (others): 826

Q2: Men completing the prep exam vs others
T-statistic: 3.25745198123116
P-value: 0.0012695902226222493
Conclusion: At a significance level of 0.05, there is a statistically significant difference in the average scores of males completing the prep exam compared to other students.


Q2： Is there a statistical difference between males who took the preparation exam and others?
I conducted an independent samples t-test comparing the average total test scores of male students who completed the preparation course with all other students. The test yielded a t-statistic of 3.25745198123116 and a p-value of 0.0012695902226222493.
As p < 0.05: This suggests a statistically significant difference between the two groups.

In [44]:
male_scores = df[df['gender'] == 'male']['total_score']
female_scores = df[df['gender'] == 'female']['total_score']

t_gender, p_gender = stats.ttest_ind(male_scores, female_scores, equal_var=False)

print("\nGender difference: Male vs Female")
print("T-statistic:", t_gender)
print("P-value:", p_gender)


Gender difference: Male vs Female
T-statistic: -4.17888598340718
P-value: 3.1861975638752864e-05


In [45]:
prep = df[df['test_preparation_course'] == 'completed']['total_score']
noprep = df[df['test_preparation_course'] == 'none']['total_score']

t_prep, p_prep = stats.ttest_ind(prep, noprep, equal_var=False)

print("\nprep course: completed vs none")
print("T-statistic:", t_prep)
print("P-value:", p_prep)


prep course: completed vs none
T-statistic: 8.594538326688614
P-value: 4.426725271318694e-17


In [46]:
results = {}
cat_cols = []
for candidate in ['gender', 'lunch', 'test_preparation_course', 'race_ethnicity']:
    if candidate in df.columns:
        cat_cols.append(candidate)

print("\nThe categorical variables to be checked:", cat_cols)

for col in cat_cols:
    groups = df.groupby(col)['total_score']
    group_lists = [g for _, g in groups]

    if len(group_lists) == 2:
        #Two groups → t-test
        t, p = stats.ttest_ind(group_lists[0], group_lists[1], equal_var=False)
        test_type = 't-test'
    else:
        # Multiple group → single factor ANOVA
        t, p = stats.f_oneway(*group_lists)
        test_type = 'ANOVA'

    results[col] = {'test': test_type, 'p_value': p}

print("\nSignificance results of each categorical variable:")
for col, res in results.items():
    print(f"{col}: {res['test']}, p = {res['p_value']}")



The categorical variables to be checked: ['gender', 'lunch', 'test_preparation_course', 'race_ethnicity']

Significance results of each categorical variable:
gender: t-test, p = 3.1861975638752864e-05
lunch: t-test, p = 1.5829690427303388e-19
test_preparation_course: t-test, p = 4.426725271318694e-17
race_ethnicity: ANOVA, p = 3.2258693161040875e-07


Q3：Are there any other differences that are of interest?
Additional t-tests and ANOVA were conducted across gender, test preparation course completion, lunch type, and race/ethnicity. For example, the comparison between students who completed the preparation course and those who did not produced a p-value of 4.43e-17, suggesting that completing the course is associated with significantly higher scores. Differences by lunch type and race/ethnicity (p = 1.58e-19 and 3.23e-07, respectively) also indicate potential performance gaps that may reflect underlying socioeconomic or demographic inequalities. Furthermore, the gender comparison yielded a p-value of 3.19e-05, suggesting that performance also differs significantly between male and female students.